# Principali modifiche alle funzioni in deWall.jl

Vecchia versione della funzione delaunayWall

In [1]:
function delaunayWall(
        P::Lar.Points,
        ax = 1,
        Pblack = Float64[],
        AFL = Array{Int64,1}[],
        tetraDict = DataStructures.Dict{Array{Int64,1},Array{Float64,1}}();
        DEBUG = false
    )::Lar.Cells

    if DEBUG @show "Delaunay Wall with parameters" P ax AFL tetraDict end

    # 0 - Data Reading and Container definition
    DT = Array{Int64,1}[]		# Delaunay Triangulation
    AFLα = Array{Int64,1}[]		# (d-1)faces intersecting the Wall
    AFLplus = Array{Int64,1}[]  # (d-1)faces in positive Wall half-space
    AFLminus = Array{Int64,1}[] # (d-1)faces in positive Wall half-space
    off = AlphaShapes.findMedian(P, ax)
    if !isempty(Pblack) Pext = [P Pblack] else Pext = copy(P) end

    # 1 - Determine first simplex (if necessary)
    if isempty(AFL)
        @assert isempty(Pblack) "delaunayWall: If AFL is empty => Pblack must be"
        @assert isempty(tetraDict) "delaunayWall: If AFL is empty => tetraDict must be"
        σ = sort(AlphaShapes.firstDeWallSimplex(P, ax, off, DEBUG = DEBUG))
        push!(DT, σ)
        AFL = AlphaShapes.simplexFaces(σ)
        AlphaShapes.updateTetraDict!(P, tetraDict, AFL, σ)
    else
        @assert !isempty(Pblack) "delaunayWall: Data missing - Pblack"
        @assert !isempty(AFL) "delaunayWall: Data missing - AFL"
        @assert !isempty(tetraDict) "delaunayWall: Data missing - tetraDict"
    end

    # 2 - Build `AFL*` according to the axis `ax` with constant term `off`
    AlphaShapes.updateAFL!(
        P, AFL, AFLα, AFLplus, AFLminus, ax, off, DEBUG = DEBUG
    )

    # 4 - Build simplex Wall
    while !isempty(AFLα)
        # if face ∈ keys(tetraDict) oppoint = tetraDict[face]
        # else Pselection = setdiff([i for i = 1 : n], face) end
        σ = AlphaShapes.findWallSimplex(
                Pext, AFLα[1], tetraDict[AFLα[1]], size(P, 2), DEBUG = DEBUG
            )
        if σ != nothing && σ ∉ DT
            push!(DT, σ)
            AFL = AlphaShapes.simplexFaces(σ)
            AlphaShapes.updateTetraDict!(P, tetraDict, AFL, σ)
            # Split σ's Faces according in semi-spaces
            AlphaShapes.updateAFL!(
                P, AFL, AFLα, AFLplus, AFLminus, ax, off, DEBUG=DEBUG
            )
        else
            @assert AlphaShapes.updatelist!(AFLα, AFLα[1]) == false "delaunayWall:
                Something unespected happends while removing a face."
        end
    end

    # 5 - Change the axis `ax` and repeat until there are no faces but exposed.
    #      A.K.A. Divide & Conquer phase.
    if !isempty(AFLminus)
        union!(DT, recursiveDelaunayWall(
            P, Pblack, tetraDict, AFLminus, ax, off, false; DEBUG = DEBUG
        ))
    end
    if !isempty(AFLplus)
        union!(DT, recursiveDelaunayWall(
            P, Pblack, tetraDict, AFLplus, ax, off, true; DEBUG = DEBUG
        ))
    end

    return DT
end


LoadError: UndefVarError: Lar not defined

Nuova versione della funzione delaunayWall

In [2]:
@timeit to "computeFirstSimplex" function computeFirstSimplex(P::Lar.Points,
    ax::Int64,
    off::Float64,
    tetraDict::Dict{Array{Int64,1},Array{Float64,1}},
    DT::Array{Array{Int64,1},1},
    DEBUG::Bool)::Array{Array{Int64,1},1}

    σ = sort(AlphaShapes.firstDeWallSimplex(P, ax, off, DEBUG = DEBUG))
    push!(DT, σ)
    AFL = @spawn AlphaShapes.simplexFaces(σ)
    AFL = fetch(AFL)
    AlphaShapes.updateTetraDict!(P, tetraDict, AFL, σ)
    return AFL
end

LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[2]:1

In [3]:
@timeit to "delaunayWall" function delaunayWall(
        P::Lar.Points,
        ax = 1,
        Pblack = Float64[],
        AFL = Array{Int64,1}[],
        tetraDict = DataStructures.Dict{Array{Int64,1},Array{Float64,1}}();
        DEBUG = false
    )::Lar.Cells

    if DEBUG @show "Delaunay Wall with parameters" P ax AFL tetraDict end

    # 0 - Data Reading and Container definition
    DT = Array{Int64,1}[]		# Delaunay Triangulation
    AFLα = Array{Int64,1}[]		# (d-1)faces intersecting the Wall
    AFLplus = Array{Int64,1}[]  # (d-1)faces in positive Wall half-space
    AFLminus = Array{Int64,1}[] # (d-1)faces in positive Wall half-space
    off =  AlphaShapes.findMedian(P, ax)

    if !isempty(Pblack) Pext = [P Pblack] else Pext = copy(P) end

    # 1 - Determine first simplex (if necessary)
    if isempty(AFL)
        @assert isempty(Pblack) "delaunayWall: If AFL is empty => Pblack must be"
        @assert isempty(tetraDict) "delaunayWall: If AFL is empty => tetraDict must be"
        AFL =  computeFirstSimplex(P,ax,off,tetraDict,DT,DEBUG)
    else
        @assert !isempty(Pblack) "delaunayWall: Data missing - Pblack"
        @assert !isempty(AFL) "delaunayWall: Data missing - AFL"
        @assert !isempty(tetraDict) "delaunayWall: Data missing - tetraDict"
    end

    # 2 - Build `AFL*` according to the axis `ax` with constant term `off`
    AlphaShapes.updateAFL!(
        P, AFL, AFLα, AFLplus, AFLminus, ax, off, DEBUG = DEBUG
    )

    # 4 - Build simplex Wall
    while !isempty(AFLα)
        # if face ∈ keys(tetraDict) oppoint = tetraDict[face]
        # else Pselection = setdiff([i for i = 1 : n], face) end

        σ =  AlphaShapes.findWallSimplex(
                Pext, AFLα[1], tetraDict[AFLα[1]], size(P, 2), DEBUG = DEBUG
            )
        if σ != nothing && σ ∉ DT
            push!(DT, σ)
            AFL = @spawn AlphaShapes.simplexFaces(σ)
            AFL = fetch(AFL)

            AlphaShapes.updateTetraDict!(P, tetraDict, AFL, σ)
            # Split σ's Faces according in semi-spaces
            AlphaShapes.updateAFL!(
                P, AFL, AFLα, AFLplus, AFLminus, ax, off, DEBUG=DEBUG
            )
        else
            @assert AlphaShapes.updatelist!(AFLα, AFLα[1]) == false "delaunayWall:
                Something unespected happends while removing a face."
        end
    end


    # 5 - Change the axis `ax` and repeat until there are no faces but exposed.
    #      A.K.A. Divide & Conquer phase.

    if !isempty(AFLminus)
        res = @spawn recursiveDelaunayWall(P, Pblack, tetraDict, AFLminus, ax, off, false; DEBUG = DEBUG)
        union!(DT, fetch(res))
    end

    if !isempty(AFLplus)
        res = @spawn recursiveDelaunayWall(P, Pblack, tetraDict, AFLplus, ax, off, true; DEBUG = DEBUG)
        union!(DT, fetch(res))
    end

    return DT
end


LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[3]:1

Vecchia versione della funzione findWallSimplex

In [4]:
function findWallSimplex(
        P::Lar.Points,
        face::Array{Int64,1},
        oppoint::Array{Float64,1},
        blackidx = size(P, 2);
        DEBUG = false
    )::Union{Array{Int64,1}, Nothing}

    if DEBUG @show "find Wall Simplex of" face oppoint end
    # Find the points in the halfspace defined by `face` that do not
    #  containsother the other point of the simplex.
    Pselection =
        AlphaShapes.oppositeHalfSpacePoints(P, P[:, face], oppoint)

    if DEBUG @show Pselection end

    # If there are no such points than the face is part of the convex hull.
    if isempty(Pselection)
        return nothing
    end

    # Find the Closest Point in the other halfspace with respect to σ
    #  according to dd-distance.
    idxbase = Pselection[ AlphaShapes.findClosestPoint(
        P[:, face], P[:, Pselection], metric = "dd"
    ) ]

    # @assert !isnothing(idxbase)
    # if isnothing(idxbase)
    # 	return nothing
    # end

    # It prevent from adding the same simplex again (cause it has been
    #  determined in a previous recursive call in the stacktrace).
    if idxbase > blackidx
        if DEBUG println("Excluding $face cause simplex already inside.") end
        return nothing
    end

    σ = sort([face; idxbase])
    if DEBUG @show "Found face" σ end

    # Check the simplex correctness
    radius, center = AlphaShapes.findRadius(P[:, σ], true)
    for i = 1 : size(P, 2)
        if Lar.norm(center - P[:, i]) < radius - 1.e-14
            # @assert i ∉ Pselection "ERROR: Numerical error
            # 	evaluating minimum radius for $σ"
            if DEBUG println("$σ discarded due to a closer point.") end
            return nothing
        end
    end

    return σ
end


LoadError: UndefVarError: Lar not defined

Nuova versione della funzione findWallSimplex

In [5]:
@timeit to "findWallSimplex" function findWallSimplex(
        P::Lar.Points,
        face::Array{Int64,1},
        oppoint::Array{Float64,1},
        blackidx = size(P, 2);
        DEBUG = false
    )::Union{Array{Int64,1}, Nothing}

    if DEBUG @show "find Wall Simplex of" face oppoint end
    # Find the points in the halfspace defined by `face` that do not
    #  containsother the other point of the simplex.
    Pselection =
        @spawn AlphaShapes.oppositeHalfSpacePoints(P, P[:, face], oppoint)
    Pselection = fetch(Pselection)

    if DEBUG @show Pselection end

    # If there are no such points than the face is part of the convex hull.
    if isempty(Pselection)
        return nothing
    end

    # Find the Closest Point in the other halfspace with respect to σ
    #  according to dd-distance.
    idxbase = @spawn Pselection[ AlphaShapes.findClosestPoint(
        P[:, face], P[:, Pselection], metric = "dd"
    ) ]
    idxbase=fetch(idxbase)

    # @assert !isnothing(idxbase)
    # if isnothing(idxbase)
    # 	return nothing
    # end

    # It prevent from adding the same simplex again (cause it has been
    #  determined in a previous recursive call in the stacktrace).
    if idxbase > blackidx
        if DEBUG println("Excluding $face cause simplex already inside.") end
        return nothing
    end

    σ = sort([face; idxbase])
    if DEBUG @show "Found face" σ end

    # Check the simplex correctness
    radius, center = AlphaShapes.findRadius(P[:, σ], true)

    for i = 1 : size(P, 2)
        if Lar.norm(center - P[:, i]) < radius - 1.e-14
            # @assert i ∉ Pselection "ERROR: Numerical error
            # 	evaluating minimum radius for $σ"
            if DEBUG println("$σ discarded due to a closer point.") end
            return nothing
        end
    end

    return σ
end


LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[5]:1

Vecchia versione della funzione firstDeWallSimplex

In [6]:
function firstDeWallSimplex(
        P::Lar.Points,
        ax::Int64,
        off::Float64;
        DEBUG = false
    )::Array{Int64,1}

    dim = size(P, 1)
    n = size(P, 2)

    if DEBUG println("Determine first Simplex with ax = $ax") end
    # the first point of the simplex is the one with coordinate `ax` maximal
    #  such that it is less than `off` (closer to α from minus)

    Pselection = findall(x -> x < off, P[ax, :])

    # it gives an error if no point are less than `off`
    #  in fact it means that all the points are located on the median,
    #  with respect to `ax`.
    @assert !isempty(Pselection) "firstDeWallSimplex: not able to build the first Delaunay
        dimplex; all the points have the same `ax` coordinate."
    newidx = Pselection[findmax(P[ax, Pselection])[2]]
    # indices will store the indices of the simplex ...
    indices = [newidx]                      #Array{Int64,1}
    # ... and `Psimplex` will store the corresponding points
    Psimplex = P[:, newidx][:,:]    #Array{Float64,2}

    # the second point must be seeken across those with coordinate `ax`
    #  grater than `off`
    Pselection = findall(x -> x > off, P[ax, :])

    for d = 1 : dim
        idxbase = AlphaShapes.findClosestPoint(Psimplex, P[:, Pselection])
        @assert !isnothing(idxbase) "firstDeWallSimplex:
            not able to determine first Delaunay Simplex"
        newidx = Pselection[idxbase]
        indices = [indices; newidx]
        Psimplex = [Psimplex P[:, newidx]]
        Pselection = [i for i = 1 : n if i ∉ indices]
    end

    # Correctness check
    radius, center = AlphaShapes.findRadius(Psimplex, true)
    for i = 1 : n
        @assert Lar.norm(center - P[:, i]) >= radius "firstDeWallSimplex:
            Unable to find first Simplex."
    end

    if DEBUG println("First Simplex = $indices") end

    return indices
end

LoadError: UndefVarError: Lar not defined

Nuova versione della funzione firstDeWallSimplex

In [7]:
@timeit to "firstDeWallSimplex" function firstDeWallSimplex(
        P::Lar.Points,
        ax::Int64,
        off::Float64;
        DEBUG = false
    )::Array{Int64,1}

    dim = size(P, 1)
    n = size(P, 2)

    if DEBUG println("Determine first Simplex with ax = $ax") end
    # the first point of the simplex is the one with coordinate `ax` maximal
    #  such that it is less than `off` (closer to α from minus)

    Pselection = @spawn findall(x -> x < off, P[ax, :])
    Pselection = fetch(Pselection)
    # it gives an error if no point are less than `off`
    #  in fact it means that all the points are located on the median,
    #  with respect to `ax`.
    @assert !isempty(Pselection) "firstDeWallSimplex: not able to build the first Delaunay
        dimplex; all the points have the same `ax` coordinate."
    newidx =  Pselection[findmax(P[ax, Pselection])[2]]
    # indices will store the indices of the simplex ...
    indices = [newidx]                      #Array{Int64,1}
    # ... and `Psimplex` will store the corresponding points
    Psimplex = P[:, newidx][:,:]    #Array{Float64,2}

    # the second point must be seeken across those with coordinate `ax`
    #  grater than `off`
    Pselection =  findall(x -> x > off, P[ax, :])

    @inbounds @simd for d = 1 : dim
        idxbase = @spawn AlphaShapes.findClosestPoint(Psimplex, P[:, Pselection])
        idxbase = fetch(idxbase)

        @assert !isnothing(idxbase) "firstDeWallSimplex:
            not able to determine first Delaunay Simplex"
        newidx = Pselection[idxbase]
        indices = [indices; newidx]
        Psimplex = [Psimplex P[:, newidx]]
        Pselection = [i for i = 1 : n if i ∉ indices]
    end

    # Correctness check
    radius, center = AlphaShapes.findRadius(Psimplex, true)

    for i = 1 : n
        @assert Lar.norm(center - P[:, i]) >= radius "firstDeWallSimplex:
            Unable to find first Simplex."
    end

    if DEBUG println("First Simplex = $indices") end

    return indices
end


LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[7]:1

Vecchia versione della funzione recursiveDelaunayWall

In [8]:
function recursiveDelaunayWall(
        P::Lar.Points,
        Pblack::Array{Float64},
        tetraDict::DataStructures.Dict{Array{Int64,1},Array{Float64,1}},
        AFL::Array{Array{Int64,1},1},
        ax::Int64,
        off::Float64,
        positive::Bool;
        DEBUG = false
    )::Lar.Cells

    #DEBUG = true

    dim, n = size(P)
    newaxis = mod(ax, dim) + 1

    if DEBUG println("Divide Plus/Minus $positive") end

    Psubset = findall(x -> (x > off) == positive, P[ax, :])
    blacklist = setdiff(unique([(keys(tetraDict)...)...]), Psubset)
    if !isempty(Pblack)
        Pblack = [Pblack P[:, blacklist]]
    else
        Pblack = P[:, blacklist]
    end

    if DEBUG println("Step In") end

    DT = AlphaShapes.delaunayWall(
            P[:, Psubset],
            newaxis,
            Pblack,
            [[findall(Psubset.==p)[1] for p in σ] for σ in AFL],
            Dict([
                [findall(Psubset.==p)[1] for p in k] => v
                for (k,v) in tetraDict
                    if k ⊆ Psubset
            ]),
            DEBUG = DEBUG
        )

    if DEBUG @show "Step Out with " DT end

    return [[Psubset[i] for i in σ] for σ in DT]
end

LoadError: UndefVarError: Lar not defined

Nuova versione della funzione recursiveDelaunayWall

In [9]:
@timeit to "findAFL" function findAFL(Psubset::Array{Int64,1},AFL::Array{Array{Int64,1},1})::Array{Array{Int64,1},1}
    return [[findall(Psubset.==p)[1] for p in σ] for σ in AFL]
end

LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[9]:1

In [10]:
@timeit to "findTetradict" function findTetradict(Psubset::Array{Int64,1},tetraDict::DataStructures.Dict{Array{Int64,1},Array{Float64,1}})::Dict{Array{Int64,1},Array{Float64,1}}
    return Dict([
        [findall(Psubset.==p)[1] for p in k] => v
         for (k,v) in tetraDict
            if k ⊆ Psubset
    ])
end

LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[10]:1

In [11]:
@timeit to "recursiveDelaunayWall" function recursiveDelaunayWall(
        P::Lar.Points,
        Pblack::Array{Float64},
        tetraDict::DataStructures.Dict{Array{Int64,1},Array{Float64,1}},
        AFL::Array{Array{Int64,1},1},
        ax::Int64,
        off::Float64,
        positive::Bool;
        DEBUG = false
    )::Lar.Cells

    #DEBUG = true

    dim, n = size(P)
    newaxis = mod(ax, dim) + 1

    if DEBUG println("Divide Plus/Minus $positive") end

    Psubset = @spawn findall(x -> (x > off) == positive, P[ax, :])
    Psubset = fetch(Psubset)

    blacklist = @spawn setdiff(unique([(keys(tetraDict)...)...]), Psubset)
    blacklist = fetch(blacklist)

    if !isempty(Pblack)
        Pblack = [Pblack P[:, blacklist]]
    else
        Pblack = P[:, blacklist]
    end

    if DEBUG println("Step In") end

    newAFL= @spawn findAFL(Psubset,AFL)
    newAFL=fetch(newAFL)
    newTetraDict= @spawn findTetradict(Psubset,tetraDict)
    newTetraDict=fetch(newTetraDict)
    DT = AlphaShapes.delaunayWall(
            P[:, Psubset],
            newaxis,
            Pblack,
            newAFL,
            newTetraDict,
            DEBUG = DEBUG
        )

    if DEBUG @show "Step Out with " DT end

    return [[Psubset[i] for i in σ] for σ in DT]
end

LoadError: LoadError: UndefVarError: @timeit not defined
in expression starting at In[11]:1